# Map Validator vs 5
This scripts reads in a CSV file with map ids and writes out an error log.  Maps and constituent  data services are scanned for public availability  
and responsiveness.  This version of the script tests the service metadata page for service responsiveness.
This may need changed in the future.

Note the two files in a subdirectory "data". You must supply maps.csv.  errorLog.csv will be written by the script - at its conclusion.

Script requires at least ArcGIS Python API v 1.4 or later and some other fun modules.

/ trbaker, November 2018

In [ ]:
# take a map id
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from IPython.display import display
import csv
import requests

#initialize
loop=0
counter=0
errorlog="Map, Map Id, Type, Layer, Layer Id \r\n"
gis = GIS("https://arcgis.com")

#read, loop, and process over list of map ids       
with open('data/maps.csv') as f:
    reader = csv.reader(f)
    for map in reader: 
        print(" ")
        print("Map number:" + str(counter))
        try:
            result = gis.content.get(map[0])
        except:
            print("Map objectID is inaccessible - not shread or missing.")
            errorlog = errorlog + "Map inaccessible,https://www.arcgis.com/home/item.html?id=" + map[0] + ",Map error" + "\r\n"
            counter=counter+1
            continue
        display(result)
        web_map_obj = WebMap(result)
        web_map_obj
        web_map_obj.layers  

        print("LAYERS:")
        loop=0  # loop counts layers in each map
        counter=counter+1  # counter gives user map counter feedback
        for lyr in web_map_obj.layers:
            loop=loop+1
            print(" ")
            #data services with single itemId (not a grouped data servcie) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if (hasattr(lyr, 'itemId')) and not (hasattr(lyr, 'url')): 
                if hasattr(lyr, 'url'):
                    print(str(loop) + " " + "itemId (" + lyr.itemId + "): " + " "  + lyr.title)
                    print(lyr.url)
                else:
                    print(str(loop) + " " + "itemId (" + lyr.itemId + "): " + " "  + lyr.title)
                    print("http://www.arcgis.com/home/item.html?id=" + lyr.itemId)
                try:
                    gis.content.get(lyr.itemId)
                    print("Test clear.")
                except:
                    print("Error accessing layer")
                    errorlog = errorlog + result.title + "," + "http://www.arcgis.com/home/item.html?id=" + lyr.itemId + ",Warning. Layer service only.," + lyr.title + "," + lyr.itemId + "\r\n"

            #map note processing.
            elif ("mapNotes" in lyr.id) or (hasattr(lyr, 'featureCollectionType')) or (not hasattr(lyr, 'url') and not hasattr(lyr, 'itemId')) : # should pick up all map note layers
                print(str(loop) + " " + "mapNotes: " + lyr.title)
                print("no URL")
            
            #all other data services ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            else:
                print(str(loop) + " " + "Layer (" + lyr.id + "): "  + lyr.title)
                if hasattr(lyr, 'url'):
                    print(lyr.url)
                else:
                    print("no map lyr.url")
                try:
                    r = requests.get(lyr.url + '?f=pjson')  #if lyr.url fails, rollover to except block below
                    resp=r.json()
                    if ("\'code\': 400" in str(resp)):
                        print('Bad URL. Code 400')
                        errorlog = errorlog + result.title + ", " + lyr.url + ",Bad URL. Code 400.," + lyr.title + ","  + "\r\n"
                    elif ("\'code\': 401" in str(resp)):
                        print ('Unauthorized. Code 401')
                        errorlog = errorlog + result.title + ", " + lyr.url + ",Unauthorized. Code 401.," + lyr.title + ","  + "\r\n"
                    elif ("\'code\': 499" in str(resp)):
                        print ('Layer not shared. Code 401')
                        errorlog = errorlog + result.title + ", " + lyr.url + ",Layer not shared. Code 499.," + lyr.title + ","  + "\r\n"
                    else:
                        print("") #empty
                except:
                    if hasattr(lyr, 'url'):
                        print('URL Get test failed.')
                        errorlog = errorlog + result.title + ", " + lyr.url + ",Warning. URL Get test failed.," + lyr.title + ","  + "\r\n"
                    else:
                        print('URL Get test failed.')
                        errorlog = errorlog + result.title + ", " + "none" + ",Warning. URL Get test failed.," + lyr.title + ","  + "\r\n"
                          
                    
# write out error log
w = open("data/errorLog.csv","w+")
w.write(errorlog)
w.close()
print(" ")
print("Complete.")